In [6]:
from torchvision import datasets
from torchvision.transforms import ToTensor

In [7]:
train_data = datasets.MNIST(
    root = "data",
    train = True,
    transform = ToTensor(),
    download = True
)

test_data = datasets.MNIST(
    root = "data",
    train = False,
    transform = ToTensor(),
    download = True
)


Failed to download (trying next):
HTTP Error 503: Service Unavailable



100.0%


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



100.0%

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



100.0%


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



100.0%

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [10]:
train_data.data.shape

torch.Size([60000, 28, 28])

In [11]:
test_data.data.shape

torch.Size([10000, 28, 28])

In [24]:
from torch.utils.data import DataLoader

loaders = {

    'train' : DataLoader(train_data,
                         batch_size= 100,
                         shuffle = True,
                         num_workers=1),
    
    'test' : DataLoader(test_data,
                         batch_size= 100,
                         shuffle = True,
                         num_workers=1)


}

In [25]:
loaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x137838ac0>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x137838940>}

In [35]:
#Structure of NN

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(1,10, kernel_size = 5)
        self.conv2 = nn.Conv2d(10,20, kernel_size = 5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320,50)
        self.fc2 = nn.Linear(50,10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)) , 2))
        x = x.view(-1,320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training = self.training)
        x = self.fc2(x)

        return F.softmax(x)

In [50]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = CNN().to(device)

optimizer = optim.Adam(model.parameters(), lr = 0.001)

loss_fn = nn.CrossEntropyLoss()

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(loaders['train']):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 20 == 0:
            print(f'train Epoch: {epoch} [{batch_idx * len(data)}/{len(loaders["train"].dataset)}] ({100. * batch_idx / len(loaders["train"]):.0f}%)]\t{loss.item():.6f}')


def test():
    model.eval()

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in loaders['test']:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_fn(output, target).item()
            pred = output.argmax(dim=1, keepdim = True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(loaders['test'].dataset)
    print(f'\nTest set: Avg loss: {test_loss:.4f}, Accuracy: {correct}/{len(loaders["test"].dataset)} 'f'({100. * correct / len(loaders["test"].dataset):.0f}%)\n')
 

In [51]:
for epoch in range(1,11):
    train(epoch)
    test()

/var/folders/7q/ft9bpswx0p12k_7_1ty12ff40000gn/T/ipykernel_1872/3005187229.py:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)


train Epoch: 1 [0/60000] (0%)]	2.304322
train Epoch: 1 [2000/60000] (3%)]	2.285463
train Epoch: 1 [4000/60000] (7%)]	2.175983
train Epoch: 1 [6000/60000] (10%)]	1.929238
train Epoch: 1 [8000/60000] (13%)]	1.830970
train Epoch: 1 [10000/60000] (17%)]	1.813639
train Epoch: 1 [12000/60000] (20%)]	1.705271
train Epoch: 1 [14000/60000] (23%)]	1.757176
train Epoch: 1 [16000/60000] (27%)]	1.727491
train Epoch: 1 [18000/60000] (30%)]	1.669785
train Epoch: 1 [20000/60000] (33%)]	1.687503
train Epoch: 1 [22000/60000] (37%)]	1.663301
train Epoch: 1 [24000/60000] (40%)]	1.634439
train Epoch: 1 [26000/60000] (43%)]	1.642160
train Epoch: 1 [28000/60000] (47%)]	1.611083
train Epoch: 1 [30000/60000] (50%)]	1.594648
train Epoch: 1 [32000/60000] (53%)]	1.668716
train Epoch: 1 [34000/60000] (57%)]	1.700540
train Epoch: 1 [36000/60000] (60%)]	1.637164
train Epoch: 1 [38000/60000] (63%)]	1.680164
train Epoch: 1 [40000/60000] (67%)]	1.623433
train Epoch: 1 [42000/60000] (70%)]	1.613434
train Epoch: 1 [44000